In [12]:
# import pymysql
# from sqlalchemy import create_engine
# import sys

# from abc import ABC, abstractmethod
# from typing import *
import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 0. 모듈의 상위 패키지를 파이썬 경로로 설정

In [1]:
import sys

In [2]:
sys.path.append("C:\Pseudonymizer\pseudonymizer")
# /path/to/your/module

#### 1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드

In [3]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [4]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "0123")

In [5]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")

In [6]:
SQL = input("SQL 쿼리문 입력변수 = ")
# SELECT * FROM DATA_FINANCE

In [7]:
queryObject.dataQueryLanguage(sql = SQL)

In [8]:
# TEST CODE
DATA_FINANCE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [13]:
pd.DataFrame(DATA_FINANCE)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,강현심,female,80,010-1826-4535,27218,충청북도 제천시 수산면 옥순봉로6길 61-32,"판잣집, 비닐하우스",6,441,61,5357418,8583708,3,5,N,0,0,N,0
1,1,서삼봉,female,30,010-1765-1467,12736,경기도 광주시 초월읍 도곡길 109-9,다세대주택,8,515,10,8870904,5328233,3,1,N,0,0,N,0
2,2,팽상아,female,53,010-6593-5091,51292,경상남도 창원시 마산회원구 회원남32길 28-1(회원동),"판잣집, 비닐하우스",1,269,17,8525658,2005663,6,5,N,0,0,N,0
3,3,박옥점,female,26,010-7714-1345,58541,전라남도 무안군 청계면 구로길 65,다가구 단독주택,7,254,10,2593007,8944424,4,1,N,0,0,N,0
4,4,윤경념,female,69,010-5360-6014,31741,충청남도 당진시 신평면 신평길 64-6,영업 겸용 단독주택,8,590,79,411525,5125357,5,3,N,0,0,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,갈재희,female,79,010-9796-1142,36447,경상북도 영덕군 달산면 청련길 10-10,다세대주택,8,868,58,835585,6144642,5,1,N,0,0,Y,0
94996,94996,림상영,male,53,010-2817-1504,26200,강원특별자치도 영월군 무릉도원면 황정길 253-11,기타,7,875,83,4691445,6000005,5,3,N,0,0,N,0
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,26,8488084,7377056,8,2,N,0,0,N,0
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,7,5109440,3625570,3,2,N,0,0,N,0


#### 2. 가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환

In [ ]:
from pseudonymizer import pseudonymizer
# from pseudonymizer import Pseudonymizer, Pseudonym
# ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)

In [ ]:
Pseudonym